In [15]:
import pandas as pd
import random

DICCIONARIO_REGIONES = {1: 'EastEurope',
                        2: 'LatAm',
                        3: 'NorthAfrica',
                        4: 'SubSaharian',
                        5: 'WesternDem',
                        6: 'EastAsia',
                        7: 'SouthEastAsia',
                        8: 'SouthAsia',
                        9: 'Pacific',
                        10: 'Caribbean'}

1. Genere una muestra de casos (1 punto). Para ello debesconsiderar:
-  Utilice los últimos 4 dígitos de su rut como semilla pseudoaleatoria.
-  Seleccione el 50% de los casos.
-  Cada base generada debe contener los siguientes elementos:
   -  El índice de desarrollo humano (undp_hdi).
   -  El nombre del país (ccodealp).
   -  La región a la que pertenece (ht_region).
   -  El PIB per cápita. (gle_cgdpc).
   -  El total de la población (imf_pop)

- Si su apellido está entre la N y la Z, escoja las siguientes variables del módulo
Salud:
    -  wef_imort: Infant mortality, deaths/1000 live births.
    -  who_alc2000: Alcohol consumption per capita (2000-).
    -  who_tobt: Current smoking of any tobacco product (Total).
    -  wdi_exph: Government expenditure on health, total(% of GDP).
    -  Guarde esta tabla procesada en un nuevo objeto.
    -  Renombre las categorías de la variableht_region denúmeros a regiones.

In [83]:
def obtener_mitad_casos_df(df_largo):
    # Primero seria obtener el rango de indexes del df
    # Después sería obtener cuantos elementos se requieren
    # Después sería obtener la cantidad de elementos, pero sin repetirse
    cincuenta_porciento_casos = int(round(df_largo / 2, 0))
    indices_a_elegir = (random.sample(
        range(df_largo), cincuenta_porciento_casos))
    return indices_a_elegir


def obtener_submuestra(df):
    SEMILLA_PSEUDOALEATORIA = '0115'
    random.seed(SEMILLA_PSEUDOALEATORIA)

    columnas_a_utilizar = ['undp_hdi', 'ccodealp', 'ht_region', 'gle_cgdpc',
                           'imf_pop', 'wef_imort', 'who_alc2000', 'who_tobt', 'wdi_exph']

    casos_a_obtener = obtener_mitad_casos_df(df.shape[0])
    df_cincuenta_casos = df.iloc[casos_a_obtener]
    df_cincuenta_columnas_a_utilizar = df_cincuenta_casos[columnas_a_utilizar]

    df_cincuenta_columnas_a_utilizar['ht_region'] = df_cincuenta_columnas_a_utilizar['ht_region'].replace(
        DICCIONARIO_REGIONES)

    return df_cincuenta_columnas_a_utilizar


In [84]:
# Con las siguientes lineas se obtuvo el diccionario que relaciona las regiones con sus codigos

# unidas = df_submuestra.merge(df_auxiliar, how='inner', on='ccodealp')
# DICCIONARIO_REGIONES = dict(unidas.groupby(by=['ht_region_x', 'ht_region_y']).sum().index)

In [85]:
df = pd.read_csv('qog_std_cs_jan18.csv')
df_auxiliar = pd.read_csv('subsample_isz.csv')

In [86]:
df_submuestra = obtener_submuestra(df)

C:\Users\Javier\AppData\Local\Temp\ipykernel_3552\3785028379.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cincuenta_columnas_a_utilizar['ht_region'] = df_cincuenta_columnas_a_utilizar['ht_region'].replace(


In [100]:
def obtener_descripciones_variables(objeto):
    descripcion_variables_continuas = df_submuestra.describe()
    print(descripcion_variables_continuas)

    variables_continuas = descripcion_variables_continuas.columns
    variables_discretas = set(df_submuestra.columns) - set(variables_continuas)

    for variable_discreta in variables_discretas:
        print(df_submuestra[variable_discreta].value_counts('%'))

In [101]:
obtener_descripciones_variables(df_submuestra)

        undp_hdi      gle_cgdpc     imf_pop   wef_imort  who_alc2000  \
count  91.000000      96.000000   75.000000   77.000000    87.000000   
mean    0.693110   13141.156818   22.851093   24.360189     4.753908   
std     0.142295   17773.468801   39.279938   24.648970     3.864448   
min     0.421000     332.269990    0.012000    1.800000     0.000000   
25%     0.594000    2866.250000    1.292000    7.200000     1.355000   
50%     0.719000    6682.109850    6.880000   14.700000     3.780000   
75%     0.793000   16388.190000   27.767500   35.700001     8.080000   
max     0.938000  108481.130000  202.768997  117.400002    15.190000   

        who_tobt   wdi_exph  
count  61.000000  94.000000  
mean   23.875410   6.768343  
std     9.307822   2.713152  
min     7.000000   1.475303  
25%    16.200001   4.849392  
50%    23.799999   6.428007  
75%    30.299999   8.287694  
max    48.900002  17.135723  
BTN    0.010309
NIC    0.010309
HRV    0.010309
FRA    0.010309
BRB    0.010309
 